In [1]:
from IPython.display import HTML
HTML(open('../style.css', 'r').read())

# Japanese IQ Test

The following puzzle is part of a recruitment test in Japan.

---

A policeman, a convict, a father and his two sons Anton and Bruno, and a mother with her two daughters Cindy and Doris have to cross a river.  On the boat there is only room for two passengers.

During the crossing, the following conditions have to be observed:
* The father is not allowed to be on a shore with one of the daughters 
  if the mother is on the other shore. 
* The mother is not allowed to be on a shore with one of the sons 
  if the father is on the other shore.
* If the criminal is not alone, then the policeman must watch him.

  However the convict can be alone on a shore, as his shackles prevent him from running away. 
* Only the father, the mother, and the policeman are able to steer the boat.

Here is the link to try it yourself:
https://flashmuseum.org/iq-%E9%81%8E%E6%B2%B3%E9%81%8A%E6%88%B2/

We are going to formulate this problem as a *symbolic transition problem*.
Then, we can solve the problem using our constraint solver.

In [2]:
import z3

We will use the following variables to encode the problem:
* `P` equals `1` if the policeman is on the western shore,
* `V` equals `1` if the villain is on the western shore,
* `M` equals `1` if the mother is on the western shore,
* `F` equals `1` if the father is on the western shore,
* `B` is the number of boys on the western shore,
* `G` is the number of girls on the western shore.

In [3]:
P  = z3.Int('P')
V  = z3.Int('V')
M  = z3.Int('M')
F  = z3.Int('F')
B  = z3.Int('B')
G  = z3.Int('G')
PX = z3.Int('PX')
VX = z3.Int('VX')
MX = z3.Int('MX')
FX = z3.Int('FX')
BX = z3.Int('BX')
GX = z3.Int('GX')

In [4]:
def start(P, V, M, F, B, G):
    return P == 1, V == 1, M == 1, F == 1, B == 2, G == 2

In [5]:
start(P, V, M, F, B, G)

(P == 1, V == 1, M == 1, F == 1, B == 2, G == 2)

In [6]:
def goal(P, V, M, F, B, G):
    return P == 0, V == 0, M == 0, F == 0, B == 0, G == 0

In [7]:
goal(P, V, M, F, B, G)

(P == 0, V == 0, M == 0, F == 0, B == 0, G == 0)

Define the condition that has to be true in every state:
* The policeman has to watch the villain (unless the villain is alone)
* The boys  must not be alone with the mother.
* The girls must not be alone with the father.

In [8]:
def invariant(P, V, M, F, B, G):
    # Policeman must watch villain unless villain is alone
    watch_condition = z3.Implies(V == P, z3.Or(P == 1, P == 0, V == 1, V == 0))
    # Boys must not be alone with mother
    boys_with_mother = z3.Implies(z3.And(M == F, B > 0), P == F)
    # Girls must not be alone with father
    girls_with_father = z3.Implies(z3.And(F == M, G > 0), P == M)
    return watch_condition, boys_with_mother, girls_with_father

In [9]:
invariant(P, V, M, F, B, G)

(Implies(V == P, Or(P == 1, P == 0, V == 1, V == 0)),
 Implies(And(M == F, B > 0), P == F),
 Implies(And(F == M, G > 0), P == M))

The function `BoatOK(P𝛼, V𝛼, M𝛼, F𝛼, B𝛼, G𝛼, P𝛽, V𝛽, M𝛽, F𝛽, B𝛽, G𝛽)` returns a formula that 
describes a crossing of the boat from the left shore to the right shore.  

It takes the following parameters:
* `P𝛼` is the number of police men on the left shore before the croosing,
* `V𝛼` is the number of villains   on the left shore before the croosing,
* `M𝛼` is the number of mothers    on the left shore before the croosing,
* `F𝛼` is the number of fathers    on the left shore before the croosing,
* `B𝛼` is the number of boys       on the left shore before the croosing,
* `G𝛼` is the number of girls      on the left shore before the croosing,
* `P𝛽` is the number of police men on the left shore after  the croosing,
* `V𝛽` is the number of villains   on the left shore after  the croosing,
* `M𝛽` is the number of mothers    on the left shore after  the croosing,
* `F𝛽` is the number of fathers    on the left shore after  the croosing,
* `B𝛽` is the number of boys       on the left shore after  the croosing,
* `G𝛽` is the number of girls      on the left shore after  the croosing.

The formula has to ensure that:
* The people move from the left shore to the right shore,
* only the policeman, the father, or the mother can steer the boat,
* there can be at most two persons on the boat.

In [10]:
def boatOK(P𝛼, V𝛼, M𝛼, F𝛼, B𝛼, G𝛼, P𝛽, V𝛽, M𝛽, F𝛽, B𝛽, G𝛽):
    # People move from left to right
    move_left_to_right = z3.And(
        Pβ == z3.If(z3.And(Pα > 0, Pβ < Pα), Pα - 1, Pα),
        Vβ == z3.If(z3.And(Vα > 0, Vβ < Vα), Vα - 1, Vα),
        Mβ == z3.If(z3.And(Mα > 0, Mβ < Mα), Mα - 1, Mα),
        Fβ == z3.If(z3.And(Fα > 0, Fβ < Fα), Fα - 1, Fα),
        Bβ == z3.If(z3.And(Bα > 0, Bβ < Bα), Bα - 1, Bα),
        Gβ == z3.If(z3.And(Gα > 0, Gβ < Gα), Gα - 1, Gα)
    )
    # Only policeman, father, or mother can steer
    steer_condition = z3.Or(Pα == Pβ, Fα == Fβ, Mα == Mβ)
    # At most two persons on the boat
    passengers = 2 - (z3.If(Pα > Pβ, 1, 0)) - (z3.If(Vα > Vβ, 1, 0)) - (z3.If(Mα > Mβ, 1, 0)) - \
                 (z3.If(Fα > Fβ, 1, 0)) - (z3.If(Bα > Bβ, 1, 0)) - (z3.If(Gα > Gβ, 1, 0))
    boat_capacity = z3.And(passengers >= 0, passengers <= 2)
    return move_left_to_right, steer_condition, boat_capacity

In [11]:
boatOK(P, V, M, F, B, G, PX, VX, MX, FX, BX, GX)

(And(PX == If(And(P > 0, PX < P), P - 1, P),
     VX == If(And(V > 0, VX < V), V - 1, V),
     MX == If(And(M > 0, MX < M), M - 1, M),
     FX == If(And(F > 0, FX < F), F - 1, F),
     BX == If(And(B > 0, BX < B), B - 1, B),
     GX == If(And(G > 0, GX < G), G - 1, G)),
 Or(P == PX, F == FX, M == MX),
 And(2 -
     If(P > PX, 1, 0) -
     If(V > VX, 1, 0) -
     If(M > MX, 1, 0) -
     If(F > FX, 1, 0) -
     If(B > BX, 1, 0) -
     If(G > GX, 1, 0) >=
     0,
     2 -
     If(P > PX, 1, 0) -
     If(V > VX, 1, 0) -
     If(M > MX, 1, 0) -
     If(F > FX, 1, 0) -
     If(B > BX, 1, 0) -
     If(G > GX, 1, 0) <=
     2))

The function `transition(P𝛼, V𝛼, M𝛼, F𝛼, B𝛼, G𝛼, P𝛽, V𝛽, M𝛽, F𝛽, B𝛽, G𝛽, i)` returns a formula
describing the $i^{\mathrm{th}}$ crossing of the river, where $i$ starts counting from $0$.
The last parameter `i` of the function `transistion` specifies the number of the crossing.

You should use of the function `boatOK`.

In [12]:
def transition(Pα, Vα, Mα, Fα, Bα, Gα, Pβ, Vβ, Mβ, Fβ, Bβ, Gβ, i):
    if i % 2 == 0:
        return boatOK(Pα, Vα, Mα, Fα, Bα, Gα, Pβ, Vβ, Mβ, Fβ, Bβ, Gβ)
    else:
        return boatOK(Pβ, Vβ, Mβ, Fβ, Bβ, Gβ, Pα, Vα, Mα, Fα, Bα, Gα)

In [13]:
transition(P, V, M, F, B, G, PX, VX, MX, FX, BX, GX, 0)

(And(PX == If(And(P > 0, PX < P), P - 1, P),
     VX == If(And(V > 0, VX < V), V - 1, V),
     MX == If(And(M > 0, MX < M), M - 1, M),
     FX == If(And(F > 0, FX < F), F - 1, F),
     BX == If(And(B > 0, BX < B), B - 1, B),
     GX == If(And(G > 0, GX < G), G - 1, G)),
 Or(P == PX, F == FX, M == MX),
 And(2 -
     If(P > PX, 1, 0) -
     If(V > VX, 1, 0) -
     If(M > MX, 1, 0) -
     If(F > FX, 1, 0) -
     If(B > BX, 1, 0) -
     If(G > GX, 1, 0) >=
     0,
     2 -
     If(P > PX, 1, 0) -
     If(V > VX, 1, 0) -
     If(M > MX, 1, 0) -
     If(F > FX, 1, 0) -
     If(B > BX, 1, 0) -
     If(G > GX, 1, 0) <=
     2))

In [14]:
transition(PX, VX, MX, FX, BX, GX, P, V, M, F, B, G, 1)

(And(PX == If(And(P > 0, PX < P), P - 1, P),
     VX == If(And(V > 0, VX < V), V - 1, V),
     MX == If(And(M > 0, MX < M), M - 1, M),
     FX == If(And(F > 0, FX < F), F - 1, F),
     BX == If(And(B > 0, BX < B), B - 1, B),
     GX == If(And(G > 0, GX < G), G - 1, G)),
 Or(P == PX, F == FX, M == MX),
 And(2 -
     If(P > PX, 1, 0) -
     If(V > VX, 1, 0) -
     If(M > MX, 1, 0) -
     If(F > FX, 1, 0) -
     If(B > BX, 1, 0) -
     If(G > GX, 1, 0) >=
     0,
     2 -
     If(P > PX, 1, 0) -
     If(V > VX, 1, 0) -
     If(M > MX, 1, 0) -
     If(F > FX, 1, 0) -
     If(B > BX, 1, 0) -
     If(G > GX, 1, 0) <=
     2))

In [15]:
def japanese_CSP(n):
    S = z3.Solver()
    # Define all variables
    P = [z3.Int(f"P[{i}]") for i in range(n+1)]
    V = [z3.Int(f"V[{i}]") for i in range(n+1)]
    M = [z3.Int(f"M[{i}]") for i in range(n+1)]
    F = [z3.Int(f"F[{i}]") for i in range(n+1)]
    B = [z3.Int(f"B[{i}]") for i in range(n+1)]
    G = [z3.Int(f"G[{i}]") for i in range(n+1)]
    
    # Define the invariants
    for i in range(n+1):
        S.add(invariant(P[i], V[i], M[i], F[i], B[i], G[i]))
    
    # Define the transitions
    for i in range(n):
        S.add(transition(P[i], V[i], M[i], F[i], B[i], G[i],
                        P[i+1], V[i+1], M[i+1], F[i+1], B[i+1], G[i+1], i))
    
    # Restrict the values of the variables
    for i in range(n+1):
        S.add(z3.And(P[i] >= 0, P[i] <= 1))
        S.add(z3.And(V[i] >= 0, V[i] <= 1))
        S.add(z3.And(M[i] >= 0, M[i] <= 1))
        S.add(z3.And(F[i] >= 0, F[i] <= 1))
        S.add(z3.And(B[i] >= 0, B[i] <= 2))
        S.add(z3.And(G[i] >= 0, G[i] <= 2))
    
    # Add start and goal constraints
    S.add(start(P[0], V[0], M[0], F[0], B[0], G[0]))
    S.add(goal(P[n], V[n], M[n], F[n], B[n], G[n]))
    
    result = str(S.check())
    if result == 'sat':
        Model = S.model()
        # Construct a single dictionary for the solution
        Solution = {}
        for i in range(n+1):
            Solution[f"P[{i}]"] = Model[P[i]].as_long()
            Solution[f"V[{i}]"] = Model[V[i]].as_long()
            Solution[f"M[{i}]"] = Model[M[i]].as_long()
            Solution[f"F[{i}]"] = Model[F[i]].as_long()
            Solution[f"B[{i}]"] = Model[B[i]].as_long()
            Solution[f"G[{i}]"] = Model[G[i]].as_long()
        return Solution
    else:
        return None

In [16]:
def find_solution():
    n = 1
    while True:
        print(n)
        Solution = japanese_CSP(n)
        if Solution is not None:
            return n, Solution
        n += 2

In [17]:
%%time
n, Solution = find_solution()
n, Solution

1
3
5
7
CPU times: total: 125 ms
Wall time: 136 ms


(7,
 {'P[0]': 1,
  'V[0]': 1,
  'M[0]': 1,
  'F[0]': 1,
  'B[0]': 2,
  'G[0]': 2,
  'P[1]': 1,
  'V[1]': 1,
  'M[1]': 0,
  'F[1]': 1,
  'B[1]': 1,
  'G[1]': 2,
  'P[2]': 1,
  'V[2]': 1,
  'M[2]': 0,
  'F[2]': 1,
  'B[2]': 1,
  'G[2]': 2,
  'P[3]': 0,
  'V[3]': 1,
  'M[3]': 0,
  'F[3]': 1,
  'B[3]': 1,
  'G[3]': 1,
  'P[4]': 0,
  'V[4]': 1,
  'M[4]': 0,
  'F[4]': 1,
  'B[4]': 1,
  'G[4]': 1,
  'P[5]': 0,
  'V[5]': 0,
  'M[5]': 0,
  'F[5]': 0,
  'B[5]': 1,
  'G[5]': 1,
  'P[6]': 0,
  'V[6]': 0,
  'M[6]': 0,
  'F[6]': 0,
  'B[6]': 1,
  'G[6]': 1,
  'P[7]': 0,
  'V[7]': 0,
  'M[7]': 0,
  'F[7]': 0,
  'B[7]': 0,
  'G[7]': 0})

## Auxiliary Code for Pretty Printing

The following code is used for printing the path that has been found.  We won't discuss the details of these functions.

In [18]:
def show_solution(Solution, n):
    for i in range(n+1):
        P = Solution[f"P[{i}]"]
        V = Solution[f"V[{i}]"]
        M = Solution[f"M[{i}]"]
        F = Solution[f"F[{i}]"]
        B = Solution[f"B[{i}]"]
        G = Solution[f"G[{i}]"]
        print(f"🕒{i}🕒 (+P)🕴️{P} (+V)👺{V} (+M)👩‍🦰{M} (+F)👨‍🦰{F} (+B)👦{B} (+G)👧{G}")

In [19]:
show_solution(Solution, n)

🕒0🕒 (+P)🕴️1 (+V)👺1 (+M)👩‍🦰1 (+F)👨‍🦰1 (+B)👦2 (+G)👧2
🕒1🕒 (+P)🕴️1 (+V)👺1 (+M)👩‍🦰0 (+F)👨‍🦰1 (+B)👦1 (+G)👧2
🕒2🕒 (+P)🕴️1 (+V)👺1 (+M)👩‍🦰0 (+F)👨‍🦰1 (+B)👦1 (+G)👧2
🕒3🕒 (+P)🕴️0 (+V)👺1 (+M)👩‍🦰0 (+F)👨‍🦰1 (+B)👦1 (+G)👧1
🕒4🕒 (+P)🕴️0 (+V)👺1 (+M)👩‍🦰0 (+F)👨‍🦰1 (+B)👦1 (+G)👧1
🕒5🕒 (+P)🕴️0 (+V)👺0 (+M)👩‍🦰0 (+F)👨‍🦰0 (+B)👦1 (+G)👧1
🕒6🕒 (+P)🕴️0 (+V)👺0 (+M)👩‍🦰0 (+F)👨‍🦰0 (+B)👦1 (+G)👧1
🕒7🕒 (+P)🕴️0 (+V)👺0 (+M)👩‍🦰0 (+F)👨‍🦰0 (+B)👦0 (+G)👧0
